### Latex macros:
$$
\newcommand{\E}{\text{E}}
\newcommand{\mbf}{\mathbf}
\newcommand{\bo}{\mathbf}
\newcommand{\bs}{\boldsymbol}
\newcommand{\Var}{\text{Var}}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\e}{\frac{1}{\sigma^2_e}}
\newcommand{\f}{\frac{1}{\sigma^2_{\alpha}}}
$$

In [7]:
macro javascript_str(s) display("text/javascript", s); end
javascript"""
    MathJax.Hub.Config({
      TeX: { equationNumbers: { autoNumber: "AMS" } }
    });
    MathJax.Hub.Queue( 
        ["resetEquationNumbers",MathJax.InputJax.TeX], 
        ["PreProcess",MathJax.Hub], 
        ["Reprocess",MathJax.Hub] 
    );
"""

# Iterative methods for solving linear systems

Consider the system of consistent linear equations: 

\begin{equation}
\label{eq:Axb}
\mathbf{Ax} = \mathbf{b},
\end{equation}
* Three iterative methods that we will use for solving the linear systems are the Jacobi method, the Gauss-Seidel Methods, and the Preconditioned Conjugate Gradient (PCCG) method. 
* These methods can be used to solve normal equations shown in the previous section and Mixed Model Equations (MME) we will covered later. 
* Consider MME, the left-hand-side (LHS) of the MME is represented by $\mathbf{A}$ and the right-hand-side (RHS) by $\mathbf{b}$. 
* The LHS of the MME is often too large to store in memory as a “fully-stored” matrix. However, $\mathbf{A}$ is often very sparse. Thus, it is may be possible to store only the non-zero elements of $\mathbf{A}$ and compute $\mathbf{Ax_n}$, using sparse matrix methods.

### The Jacobi method

In the Jacobi method, the solution at iteration $t+1$ can be written as: 
\begin{equation}
  \label{eq:jacobi}
\mathbf{x}^{t+1} = \mathbf{D}^{−1}(\mathbf{b} − \mathbf{A}\mathbf{x}^t) + \mathbf{x}^t,
\end{equation}
where $\mathbf{D}$ is the diagonal component of $\mathbf{A}$.

#### Weighted Jacobi algorithm

Convergence can often be improved by using a weighted Jacobi approach:
\begin{equation}
\mathbf{x}^{t+1*} = \alpha\mathbf{x}^{t+1} + (1 − \alpha)\mathbf{x}^{t*}   \label{eq:jacobi_modified}
\end{equation}
for $0 < \alpha < 1$.

Straightforward application of the Jacobi method to solve the normal equations or mixed model equations
would require first computing the left-hand-side (LHS) and right-hand-side (RHS) of the normal equations or mixed model equations, and then using (\ref{eq:jacobi_modified}) until convergence.

#### Numerical Example

In [56]:
using LinearAlgebra, Distributions, DataFrames

In [57]:
data = DataFrame(x=[1,1,2,2,2,2,3,3,4,1],y=[1.1,1.2,1.9,1.2,2.0,1.7,1.0,1.7,1.1,1.7])
n = size(data,1)
p = length(unique(data[:,:x]))
X = zeros(n,p);
for i = 1:n
    j = data[:,:x][i]
    X[i,j] = 1.0
end
X = [ones(n) X];
y = data[:,:y];
lhs = X'X;
rhs = X'y;

In [58]:
lhs

5×5 Array{Float64,2}:
 10.0  3.0  4.0  2.0  1.0
  3.0  3.0  0.0  0.0  0.0
  4.0  0.0  4.0  0.0  0.0
  2.0  0.0  0.0  2.0  0.0
  1.0  0.0  0.0  0.0  1.0

In [71]:
D = Diagonal(lhs)
A = lhs + I*0.0
b = rhs
x = ones(5)
for i = 1:20
    x=inv(D)*(b-A*x)+x
    println(x)
end

[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]
[0.46, 0.333333, 0.7, 0.35, 0.1]
[1.0, 0.873333, 1.24, 0.89, 0.64]


In [11]:
[A*x b]

5×2 Array{Float64,2}:
 20.0   14.6
  5.62   4.0
  8.96   6.8
  3.78   2.7
  1.64   1.1

#### Function for Weighed Jacobi Iteration

In [13]:
function WJacobi(A,x,b,p=0.7;tolerance=0.000001,printout_frequency=10,maxiter=1000) #optional arguments keyword arguments
    n       = size(A,1)        #number of linear equations
    D       = diag(A)
    error   = b - A*x
    diff    = sum(error.^2)/n
    
    iter    = 0
    while (diff > tolerance) && (iter < maxiter)
        iter   += 1
        error   = b - A*x
        x_temp  = error./D + x
        x       = p*x_temp + (1-p)*x
        diff    = sum(error.^2)/n
        
        if iter%printout_frequency == 0
            println(iter," ",diff)
        end
    end
    return x
end

WJacobi (generic function with 2 methods)

In [15]:
starting_value = zeros(size(lhs,2))
sol=WJacobi(lhs,starting_value,rhs,0.7)
[A*sol A*(lhs\rhs)]

10 3.8029144498142352e-6


5×2 Array{Float64,2}:
 14.6006   14.6
  4.00018   4.0
  6.80024   6.8
  2.70012   2.7
  1.10006   1.1

#### Why does weighted Jacobi work better?

Equation (\ref{eq:Axb}) can be rearranged as follows to get 

\begin{align}
\bo{Ax} &= \bo{b} \\
(\bo{D} + \bo{E})\bo{x} &= \bo{b}\\
\bo{D}\bo{x} &= -\bo{E}\bo{x} + \bo{b}\\
\bo{x} &= -\bo{D}^{-1}\bo{E}\bo{x} + \bo{D}^{-1}\bo{b}\\
\bo{x} &= \bo{B}\bo{x} + \bo{z}, 
\label{eq:xBXz}
\end{align}

where $\bo{B} = -\bo{D}^{-1}\bo{E}$ and $\bo{z} = \bo{D}^{-1}\bo{b}$. Functional iteration using (\ref{eq:xBXz}) results in the Jacobi method:

\begin{equation}
\label{eq:Jacobi1}
\bo{x}^{(t+1)} = \bo{B}\bo{x}^{t} + \bo{z}, 
\end{equation}

Let $\bo{x}$ be a solution for (\ref{eq:Axb}). Then, (\ref{eq:Jacobi1}) can be written as

\begin{equation}
\label{eq:Jacobi2}
\bo{x}^{(t+1)} = \bo{B}(\bo{x} + \bo{e}^{t}) + \bo{z}, 
\end{equation}

where $\bo{e}^{t} = \bo{x}^t - \bo{x}$. But, from (\ref{eq:xBXz}), $\bo{x} = \bo{B}\bo{x} + \bo{z}$, and therefore (\ref{eq:Jacobi2}) can be written as
\begin{equation}
\label{eq:Jacobi3}
\bo{x}^{(t+1)} = \bo{x} + \bo{B}\bo{e}^{t}, 
\end{equation}
or
\begin{equation}
\label{eq:Jacobi4}
\bo{e}^{(t+1)} = \bo{B}\bo{e}^{t},
\end{equation}

which implies 
\begin{equation}
\label{eq:Jacobi5}
\bo{e}^{(t+1)} = \bo{B}^{t+1}\bo{e}^{0}.
\end{equation}

For the matrix $\bo{B}$ in (\ref{eq:Jacobi4}), let $\bs{\Lambda}$ be a diagonal matrix with its eigen values and $\bo{R}$ a matrix with the corresponding eigen vectors. Provided that the matrix $\bo{R}$ is non-singular, it can be shown that $\bo{B}^t = \bo{R}\bo{\Lambda}^{t}\bo{R}^{-1}$, and thus
\begin{equation}
\label{eq:Jacobi6}
\bo{e}^{(t)} = \bo{R}\bo{\Lambda}^{t}\bo{R}^{-1}\bo{e}^{0}.
\end{equation}

#### Eigen values and Eigen vectors of $\bo{B}$

Can see below that $\bo{B}$ has two non-zero eigen values equal to 1 and -1. Thus, (\ref{eq:Jacobi6}) does not converge.  

In [16]:
A

5×5 Array{Float64,2}:
 10.0  3.0  4.0  2.0  1.0
  3.0  3.0  0.0  0.0  0.0
  4.0  0.0  4.0  0.0  0.0
  2.0  0.0  0.0  2.0  0.0
  1.0  0.0  0.0  0.0  1.0

In [17]:
E = A - D

5×5 Array{Float64,2}:
 0.0  3.0  4.0  2.0  1.0
 3.0  0.0  0.0  0.0  0.0
 4.0  0.0  0.0  0.0  0.0
 2.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0

In [18]:
E + D == A

true

In [19]:
B = -inv(D)*E

5×5 Array{Float64,2}:
 -0.0  -0.3  -0.4  -0.2  -0.1
 -1.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -0.0  -0.0  -0.0  -0.0

In [20]:
d,R = eigen(B)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
5-element Array{Float64,1}:
  1.0               
 -1.0000000000000002
 -0.0               
 -0.0               
 -0.0               
eigenvectors:
5×5 Array{Float64,2}:
  0.447214  -0.447214  -1.16153e-17   1.34094e-17  -3.20829e-18
 -0.447214  -0.447214  -0.621287     -0.440536     -0.171607   
 -0.447214  -0.447214   0.676061     -0.0910583    -0.0773749  
 -0.447214  -0.447214  -0.280128      0.888451     -0.0773749  
 -0.447214  -0.447214  -0.280128     -0.0910583     0.97907    

In [21]:
Λ = diagm(0=>d)

5×5 Array{Float64,2}:
 1.0   0.0  0.0  0.0  0.0
 0.0  -1.0  0.0  0.0  0.0
 0.0   0.0  0.0  0.0  0.0
 0.0   0.0  0.0  0.0  0.0
 0.0   0.0  0.0  0.0  0.0

In [22]:
Ri = inv(R)

5×5 Array{Float64,2}:
  1.11803      -0.33541   -0.447214  -0.223607  -0.111803 
 -1.11803      -0.33541   -0.447214  -0.223607  -0.111803 
 -7.20078e-17  -0.580115   0.838839  -0.206979  -0.0517447
 -7.02935e-17  -0.566304  -0.202051   0.818869  -0.0505128
 -6.51744e-17  -0.525063  -0.187337  -0.187337   0.899737 

In [23]:
e0 = [1,0,0,0,0]
Ri*e0

5-element Array{Float64,1}:
  1.118033988749895    
 -1.1180339887498951   
 -7.200778190572192e-17
 -7.029350243045226e-17
 -6.517437471516372e-17

In [24]:
R*Λ

5×5 Array{Float64,2}:
  0.447214  0.447214  0.0  0.0  0.0
 -0.447214  0.447214  0.0  0.0  0.0
 -0.447214  0.447214  0.0  0.0  0.0
 -0.447214  0.447214  0.0  0.0  0.0
 -0.447214  0.447214  0.0  0.0  0.0

In [25]:
R*Λ^2

5×5 Array{Float64,2}:
  0.447214  -0.447214  0.0  0.0  0.0
 -0.447214  -0.447214  0.0  0.0  0.0
 -0.447214  -0.447214  0.0  0.0  0.0
 -0.447214  -0.447214  0.0  0.0  0.0
 -0.447214  -0.447214  0.0  0.0  0.0

In [26]:
R*Λ^100*Ri*e0

5-element Array{Float64,1}:
 1.0000000000000113    
 1.1108967702578393e-14
 1.1213437802440985e-14
 1.1213437802440985e-14
 1.1213437802440985e-14

In [27]:
R*Λ^101*Ri*e0

5-element Array{Float64,1}:
 -1.1523754571596893e-14
 -1.0000000000000115    
 -1.0000000000000113    
 -1.0000000000000113    
 -1.0000000000000113    

In [28]:
R*Λ^102*Ri*e0

5-element Array{Float64,1}:
 1.0000000000000115    
 1.1295157764071938e-14
 1.1399627863934529e-14
 1.1399627863934529e-14
 1.1399627863934529e-14

### Weighted Jacobi

The weighted Jacobi algorithm can be obtained by writing $\bo{x}$ as

$$
\bo{x} = \alpha\bo{x} + (1-\alpha)\bo{x},
$$
and substituting (\ref{eq:xBXz}) for the first term on right to get

$$
\bo{x} = \alpha(\bo{B}\bo{x} + \bo{z}) + (1-\alpha)\bo{x}.
$$

Using this equation for functional iteration, the weighted Jacobi algorithm can be written as:

\begin{align}
\bo{x}^{(t+1)} &= \alpha(\bo{B}\bo{x}^t + \bo{z}) + (1-\alpha)\bo{x}^t \\
               &= [(1-\alpha)\bo{I} + \alpha\bo{B}]\bo{x}^t + \alpha\bo{z}\\
               &= \bo{C}\bo{x}^t + \alpha\bo{z},
\label{eq:wjacobi1}               
\end{align} 
where $\bo{C} = (1-\alpha)\bo{I} + \alpha\bo{B}$.

As reasoned before, (\ref{eq:wjacobi1}) can be written as

\begin{equation}
\label{eq:Cte0}
\bo{x}^{(t+1)} = \bo{x} + \bo{C}\bo{e}^{t}, 
\end{equation}
or
\begin{align}
\bo{e}^{(t+1)} &= \bo{C}\bo{e}^{t}\\
               &= \bo{R}\bs{\Lambda}^{t+1}\bo{R}^{-1}\bo{e}^0,
\end{align}
where $\bs{\Lambda}$ and $\bo{R}$ contain the eigen values and eigen vectors of $\bo{C}$.  

Can see below that the largest eigen value of $\bo{C}$ is 1.0, and the rest, in absolute value, are smaller than 1. This implies that the system of equations given by (\ref{eq:Axb}) is singular and has an infinite number of solutions.

### Exercise 1

1. Use the `WJocobi` function given above to get a solution to the system (\ref{eq:Axb}) for the Matrix $\bo{A}$ and the vector $\bo{b}$ of this numerical example. 

2. Using the solution obtained above in equation (\ref{eq:Cte0}), obtain two other solutions to (\ref{eq:Axb}). 

#### Eigen values and Eigen vectors of $\bo{C}$

In [29]:
α = 0.7
C = I*(1-α) + B*α 

5×5 Array{Float64,2}:
  0.3  -0.21  -0.28  -0.14  -0.07
 -0.7   0.3   -0.0   -0.0   -0.0 
 -0.7  -0.0    0.3   -0.0   -0.0 
 -0.7  -0.0   -0.0    0.3   -0.0 
 -0.7  -0.0   -0.0   -0.0    0.3 

In [30]:
d,R = eigen(C)
Λ = diagm(0=>d)

5×5 Array{Float64,2}:
 1.0   0.0  0.0  0.0  0.0
 0.0  -0.4  0.0  0.0  0.0
 0.0   0.0  0.3  0.0  0.0
 0.0   0.0  0.0  0.3  0.0
 0.0   0.0  0.0  0.0  0.3

In [31]:
Ri = inv(R)

5×5 Array{Float64,2}:
  1.11803      -0.33541  -0.447214   -0.223607  -0.111803
 -1.11803      -0.33541  -0.447214   -0.223607  -0.111803
 -2.22045e-16   0.87178  -0.0484322  -0.621391  -0.201956
  0.0           0.0      -0.904379    0.452189   0.452189
  0.0           0.0       0.0        -0.747524   0.747524

In [32]:
e0 = [1,0,0,0,0]
h = Ri*e0

5-element Array{Float64,1}:
  1.1180339887498945   
 -1.1180339887498947   
 -2.220446049250313e-16
  0.0                  
  0.0                  

In [33]:
R*Λ^100

5×5 Array{Float64,2}:
  0.447214  -7.18645e-41  -1.04142e-69  -3.06489e-69  -7.14908e-70
 -0.447214  -7.18645e-41   4.13825e-53   2.05786e-53  -8.16261e-54
 -0.447214  -7.18645e-41  -1.77354e-53  -3.32424e-53   8.42287e-54
 -0.447214  -7.18645e-41  -1.77354e-53   2.37445e-53  -2.60494e-53
 -0.447214  -7.18645e-41  -1.77354e-53   2.37445e-53   4.28952e-53

In [34]:
R*Λ^100*Ri*e0

5-element Array{Float64,1}:
  0.5                
 -0.4999999999999996 
 -0.49999999999999983
 -0.49999999999999983
 -0.49999999999999983

In [35]:
R*Λ^101*Ri*e0

5-element Array{Float64,1}:
  0.5                
 -0.4999999999999996 
 -0.49999999999999983
 -0.49999999999999983
 -0.49999999999999983

In [36]:
R[:,1]*h[1]

5-element Array{Float64,1}:
  0.5                
 -0.4999999999999996 
 -0.49999999999999983
 -0.49999999999999983
 -0.49999999999999983

### The Gauss–Seidel method

The Gauss–Seidel method can be obtained by first writing (\ref{eq:Axb}) as

\begin{align} 
\bo{Ax} &= \bo{b} \\
(\bo{L}+\bo{D}+\bo{U})\bo{x} &= \bo{b} \\
\bo{D}\bo{x} &= \bo{b} - \bo{L}\bo{x} - \bo{U}\bo{x} \\
\bo{x} &= \bo{D}^{-1}(\bo{b} - \bo{L}\bo{x} - \bo{U}\bo{x}), 
\label{gs}
\end{align}
where $\mathbf{D}$ is a diagonal matrix with the diagonal elements of $\mathbf{A}$, and  $\mathbf{L}$ and $\mathbf{U}$ are matrices with the strictly lower and upper triangular components of $\mathbf{A}$.

Functional iteration using (\ref{gs}) as:

\begin{equation}
\label{eq:GS}
\bo{x}^{(t+1)} = \bo{D}^{-1}(\bo{b} - \bo{L}\bo{x}^{(t+1)} - \bo{U}\bo{x}^t)
\end{equation}

is the Gauss-Seidel algorithm, which is equivalent to

\begin{equation}
\label{eq:GS1}
\bo{x}^{(t+1)} = (\bo{D + L})^{-1}(\bo{b} - \bo{U}\bo{x}^t).
\end{equation}

The Gauss-Seidel algorithm given by (eq:GS) only requires the inverse of a diagonal matrix, and thus it is used for iteration. As we will see below, the second form given by (\ref{eq:GS1}) is useful to study the convergence properties of the algorithm.

To study the convergence of Gauss-Seidel, (\ref{eq:GS1}) can be written as

\begin{equation}
\label{eq:GS2}
\bo{x}^{(t+1)} = \bo{B}\bo{x}^t + \bo{z},
\end{equation}

for $\bo{B} = -(\bo{D + L})^{-1}\bo{U}$ and $\bo{z} = (\bo{D + L})^{-1}\bo{b}$. Given $\bo{x}$ is a solution to (\ref{eq:Axb}) and $\bo{e}^t = \bo{x}^t - \bo{x}$, (\ref{eq:GS2}) can be written as:

\begin{align}
\label{eq:GS3}
\bo{x}^{(t+1)} &= \bo{x} + \bo{B}\bo{e}^{t}   \\
               &= \bo{x} + \bo{B}^t\bo{e}^{0} \\
               &= \bo{x} + \bo{R}\bs{\Lambda}^t\bo{R}^{-1}\bo{e}^{0}
\end{align}

Can see below that the matrix $\bo{B}$ for Gauss-Seidel in this example has only one non-zero eigen value, which is equal to 1.0. Thus, in this example, Gauss-Seidel will converge in one iteration.  

In [73]:
L = LowerTriangular(A-D)
U = UpperTriangular(A-D)
B = -inv(D+L)*U

5×5 Array{Float64,2}:
 0.0  -0.3  -0.4  -0.2  -0.1
 0.0   0.3   0.4   0.2   0.1
 0.0   0.3   0.4   0.2   0.1
 0.0   0.3   0.4   0.2   0.1
 0.0   0.3   0.4   0.2   0.1

In [74]:
d,R = eigen(B)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
5-element Array{Float64,1}:
  0.0                   
  1.1102230246251565e-16
  1.0000000000000002    
 -2.6020852139652106e-17
  1.5407439555097887e-33
eigenvectors:
5×5 Array{Float64,2}:
 1.0  -0.430735  -0.447214  -0.662932   -1.0        
 0.0  -0.72465    0.447214   0.0710403   4.76929e-18
 0.0   0.310564   0.447214  -0.390077   -5.34648e-17
 0.0   0.310564   0.447214   0.449062   -8.47158e-18
 0.0   0.310564   0.447214   0.449062    2.16494e-16

In [75]:
Λ = Diagonal(d) 

5×5 Diagonal{Float64,Array{Float64,1}}:
 0.0   ⋅            ⋅     ⋅            ⋅         
  ⋅   1.11022e-16   ⋅     ⋅            ⋅         
  ⋅    ⋅           1.0    ⋅            ⋅         
  ⋅    ⋅            ⋅   -2.60209e-17   ⋅         
  ⋅    ⋅            ⋅     ⋅           1.54074e-33

In [76]:
x0 = zeros(5)
x1 = B*x0 + inv(D+L)*b

5-element Array{Float64,1}:
  1.4600000000000002 
 -0.12666666666666693
  0.23999999999999977
 -0.1100000000000001 
 -0.3600000000000001 

In [77]:
[A*x1 b]

5×2 Array{Float64,2}:
 14.6  14.6
  4.0   4.0
  6.8   6.8
  2.7   2.7
  1.1   1.1

### Function for Gauss-Seidel Iteration

In [43]:
function GaussSeidel(A,x,b;tolerance=0.000001,printout_frequency=10,maxiter=1000)
    n = size(A,1)
    for i = 1:n
        x[i] = (b[i] - A[:,i]'x)/A[i,i] + x[i]
    end
    error = b - A*x
    diff  = sum(error.^2)/n
    
    iter  = 0
    while (diff > tolerance) & (iter < maxiter)
        iter += 1
        for i = 1:n
            x[i] = (b[i] - A[:,i]'x)/A[i,i] + x[i]
        end
        
        error = b - A*x
        diff  = sum(error.^2)/n
        if iter%printout_frequency == 0
            println(iter," ",diff)
        end
    end
    return x
end

GaussSeidel (generic function with 1 method)

In [78]:
n,p = 10_000, 1_000
M = randn(n,p)
A = M'M 
b = M'randn(n);

In [79]:
x = zeros(p)
sol = GaussSeidel(A,x,b,printout_frequency=1)

1 15.5423898672392
2 0.747481922735545
3 0.042802378680376875
4 0.0026518250989305573
5 0.00017253928476386742
6 1.140861459528081e-5
7 7.558592067752963e-7


1000-element Array{Float64,1}:
 -0.014016639896975741  
  0.0031046974915040857 
 -0.009291592858652975  
 -0.01097627788553268   
  0.010980688316261922  
 -0.013317530832179868  
 -0.02011604124110173   
  0.009936201758740795  
 -0.0009044914866329442 
  0.0010000821501196671 
  0.011620713871942889  
 -0.00337877937744122   
 -0.015115308956675736  
  ⋮                     
 -0.010537567017081681  
 -0.004648442208199943  
  0.020448021411020513  
  0.00438536858329927   
  0.009505758870220626  
  0.008073089662709018  
 -0.005254177518928358  
  0.00013618210633380685
  0.009275968123015241  
 -0.009806557425979332  
 -0.00584873465131198   
 -0.007261014099933419  

In [47]:
[A*sol b] 

1000×2 Array{Float64,2}:
   24.6146     24.6139 
  -54.303     -54.3033 
 -132.881    -132.881  
   59.4318     59.4321 
  -26.7313    -26.7315 
  -86.9356    -86.9359 
 -251.904    -251.904  
  -83.6643    -83.6639 
  -42.8851    -42.8847 
    3.04585     3.04535
   -6.59614    -6.59645
  -81.3532    -81.352  
   17.3088     17.3082 
    ⋮                  
   27.8247     27.8247 
  -65.5744    -65.5744 
  168.099     168.099  
   59.6568     59.6568 
 -105.059    -105.059  
  121.764     121.764  
   -5.90031    -5.90032
  -32.8158    -32.8158 
  -46.6275    -46.6275 
  -18.1414    -18.1414 
  -52.2655    -52.2655 
  -70.7796    -70.7796 

### Exercise 2

Modify the Gauss-Seidel function given above to for a weighted Gauss-Seidel algorithm. Compare convergence of the new algorithm with that given above for a range of weights between 0 and 1.

In [80]:
D = Diagonal(A)
L = LowerTriangular(A-D)
U = UpperTriangular(A-D)
B = -inv(D+L)*U
d,R = eigen(B)
Λ = Diagonal(d);

In [90]:
maximum(B^10)

3.0184378539825316e-7

### [Conjugate Gradient Method](3.4.ConjugateGradient.ipynb)